# load file

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Dir_G.gml to Dir_G.gml
User uploaded file "Dir_G.gml" with length 942 bytes


In [ ]:
pip install networkx

In [ ]:
import networkx as nx


In [ ]:
file_path = "/content/drive/MyDrive/My_paper/Dir_G.gml"

In [ ]:
file_path = "Dir_G.gml"

In [ ]:
import networkx as nx

# Load the graph from GML file
graph = nx.read_gml(file_path)


## Create and Edit Graph

In [ ]:
import networkx as nx

# Load the graph from GML file
graph_ed = nx.read_gml(file_path)

# Add a new node
#graph_ed.add_node("new_node")

# Add a new edge
graph_ed.add_edge("H1", "S4")

# Save the updated graph
nx.write_gml(graph_ed, "updated_graph.gml")

In [ ]:
graph = nx.read_gml("updated_graph.gml")

# Last Version

### class documettion

Conclusion:

The NetworkWithDataFlow class provides functionality for managing a network graph with data flow. It allows adding nodes, modifying nodes, adding edges, and managing data flow paths within the graph. It also provides methods for saving and loading the graph structure, printing the graph structure, finding alternate paths for flows, and finding safe update paths.

The class utilizes the networkx library for graph operations and pickle for serialization. The main methods and attributes of the class include:

**__init__()**: Initializes the class by creating an empty directed graph, data flow list, and sets for ingress, egress, and switch nodes.

**add_node(node, node_type)**: Adds a node to the graph with the specified node type (ingress, egress, or switch).

**modify_node(node, node_type)**: Modifies the type of an existing node in the graph.

**add_edge(source, target)**: Adds an edge between two nodes in the graph.

**modify_data_flow(flow_index, new_path)**: Modifies an existing data flow path at the specified index.

**add_data_flow(new_path)**: Adds a new data flow path to the graph.

**save_structure(filename)**: Saves the current graph structure, ingress nodes, egress nodes, switch nodes, and data flow paths to a file using pickle.

**load_structure(filename)**: Loads the graph structure, ingress nodes, egress nodes, switch nodes, and data flow paths from a file.

**print_structure()**: Prints the nodes, edges, data flows, ingress nodes, egress nodes, and switch nodes of the graph.

**print_alternate_paths_for_flows()**: Prints the alternate paths for each data flow in the graph.

**load_graph_from_gml(filename)**: Loads the graph structure from a GML file.

**save_graph_to_gml(filename)**: Saves the current graph structure to a GML file.

**find_alternate_paths(path)**: Finds alternate paths for a given data flow path in the graph.

**find_safe_update(flow_index)**: Finds and prints the safe update paths for a given data flow index that do not conflict with other flows.

From this point on, you can continue working with the NetworkWithDataFlow class by calling its methods or implementing additional functionality as per your requirements.

## Final NetworkWithDataFlow Class

In [ ]:
import pickle
import networkx as nx

class NetworkWithDataFlow:
    def __init__(self):
        self.graph = nx.DiGraph()
        self.data_flow = []
        self.ingress_nodes = set()
        self.egress_nodes = set()
        self.switch_nodes = set()

    def add_node(self, node, node_type):
        self.graph.add_node(node)
        if node_type == 'ingress':
            self.ingress_nodes.add(node)
        elif node_type == 'egress':
            self.egress_nodes.add(node)
        elif node_type == 'switch':
            self.switch_nodes.add(node)

    def modify_node(self, node, node_type):
        if node in self.graph.nodes:
            if node_type == 'ingress':
                self.ingress_nodes.add(node)
                self.egress_nodes.discard(node)
                self.switch_nodes.discard(node)
            elif node_type == 'egress':
                self.egress_nodes.add(node)
                self.ingress_nodes.discard(node)
                self.switch_nodes.discard(node)
            elif node_type == 'switch':
                self.switch_nodes.add(node)
                self.ingress_nodes.discard(node)
                self.egress_nodes.discard(node)
        else:
            print(f"Node '{node}' does not exist in the graph.")

    def add_edge(self, source, target):
        self.graph.add_edge(source, target)

    def _check_conflict_between(self, path1, path2):
        edges1 = set(zip(path1[:-1], path1[1:]))
        edges2 = set(zip(path2[:-1], path2[1:]))
        return bool(edges1.intersection(edges2))

    def modify_data_flow(self, flow_index, new_path):
        if flow_index < len(self.data_flow):
            if nx.is_simple_path(self.graph, new_path):
                if self._validate_flow_nodes(new_path):
                    conflicting_paths = [
                        path for i, path in enumerate(self.data_flow) if i != flow_index and self._check_conflict_between(path, new_path)
                    ]
                    if conflicting_paths:
                        print("Conflict detected with the following paths:")
                        for path in conflicting_paths:
                            print(path)
                        print("Cannot modify data flow due to conflicts.")
                    else:
                        self.data_flow[flow_index] = new_path
                        print("Data flow modified successfully.")
                else:
                    print("Invalid data flow path. Incorrect node types.")
            else:
                print("Invalid data flow path. Not a simple path in the graph.")
        else:
            print("Invalid data flow index.")

    def add_data_flow(self, new_path):
        if nx.is_simple_path(self.graph, new_path):
            if self._validate_flow_nodes(new_path):
                conflicting_paths = [
                    path for path in self.data_flow if self._check_conflict_between(path, new_path)
                ]
                if conflicting_paths:
                    print("Conflict detected with the following paths:")
                    for path in conflicting_paths:
                        print(path)
                    print("Cannot add data flow due to conflicts.")
                else:
                    self.data_flow.append(new_path)
                    print("Data flow added successfully.")
            else:
                print("Invalid data flow path. Incorrect node types.")
        else:
            print("Invalid data flow path. Not a simple path in the graph.")




    def _validate_flow_nodes(self, path):
        if len(path) < 2:
            return False

        if path[0] not in self.ingress_nodes:
            return False

        if path[-1] not in self.egress_nodes:
            return False

        for node in path[1:-1]:
            if node not in self.switch_nodes:
                return False

        return True

    def save_structure(self, filename):
        data = {
            'graph': self.graph,
            'ingress_nodes': self.ingress_nodes,
            'egress_nodes': self.egress_nodes,
            'switch_nodes': self.switch_nodes,
            'data_flow': self.data_flow
        }

        with open(filename, 'wb') as f:
            pickle.dump(data, f)

    def load_structure(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)

        self.graph = data['graph']
        self.ingress_nodes = data['ingress_nodes']
        self.egress_nodes = data['egress_nodes']
        self.switch_nodes = data['switch_nodes']
        self.data_flow = data['data_flow']

    def print_structure(self):
        print("nodes:\n", self.graph.nodes)
        print("edges:\n", self.graph.edges)
        print("flows:\n", self.data_flow)
        print("ingress nodes:\n", self.ingress_nodes)
        print("egress nodes:\n", self.egress_nodes)
        print("switches:\n", self.switch_nodes)

    def print_alternate_paths_for_flows(self):
        for flow_index, flow_path in enumerate(self.data_flow):
            print(f"Flow {flow_index}:")
            alternate_paths = self.find_alternate_paths(flow_path)
            for alt_path in alternate_paths:
                print(alt_path)
            print()

    def load_graph_from_gml(self, filename):
        self.graph = nx.read_gml(filename)

    def save_graph_to_gml(self, filename):
        nx.write_gml(self.graph, filename)

    def find_alternate_paths(self, path):
        source = path[0]
        destination = path[-1]
        paths = list(nx.all_simple_paths(self.graph, source=source, target=destination))
        alternate_paths = [p for p in paths if p != path]
        return alternate_paths


    def find_safe_update(self, flow_index):
        # First, find an alternate path for the given flow_index
        flow_path = self.data_flow[flow_index]
        alternate_paths = self.find_alternate_paths(flow_path)
        print(f"Alternate paths for flow {flow_index}: {flow_path}\n")
        for path in alternate_paths:
            print(path)
        print()

        # Second, find and print the safe update paths that do not conflict with other flows
        safe_update_paths = []
        for i, path in enumerate(alternate_paths):
            conflicting_paths = [f for j, f in enumerate(self.data_flow) if j != flow_index and self._check_conflict_between(f, path)]
            if not conflicting_paths:
                safe_update_paths.append(path)

        print("Safe update paths:")
        for path in safe_update_paths:
            print(path)
        print()

        return safe_update_paths

## Load graph and flows



In [ ]:
# Example usage:
network = NetworkWithDataFlow()

# Load the graph from a GML file
network.load_graph_from_gml('/content/drive/MyDrive/My_paper/Dir_G.gml')



# it is nessesary to modiify node as {ingress, egress and switch}
# because when you want to add flow it will check first node be ingress and last node egress
# Modify the type of nodes
network.modify_node('H1', 'ingress')

network.modify_node('S1', 'switch')
network.modify_node('S2', 'switch')
network.modify_node('S3', 'switch')
network.modify_node('S4', 'switch')
network.modify_node('S5', 'switch')
network.modify_node('S6', 'switch')

network.modify_node('H2', 'egress')
network.modify_node('H3', 'egress')


# Add a data flow path
network.add_data_flow(['H1', 'S2', 'S5', 'H3'])
network.add_data_flow(['H1', 'S3', 'S6', 'H3'])
network.add_data_flow(['H1', 'S1', 'S4', 'H2'])

#deleberatly add some fake flow to check validity
network.add_data_flow([ 'S3', 'S6', 'H3']) # ingress
network.add_data_flow(['H1', 'S3', 'S6']) # egress
network.add_data_flow(['H1', 'S1', 'H3']) # has not path

# Access the loaded data
network.print_structure()

path = ['H1', 'S1', 'S4', 'H3'] # Replace with the actual path you want to find alternate paths for
alternate_paths = network.find_alternate_paths(path)
print("Alternate Paths:")
for alt_path in alternate_paths:
    print(alt_path)

network.print_alternate_paths_for_flows()

# Save the network structure
network.save_structure('/content/drive/MyDrive/My_paper/net_fromFilePlusflows.pkl')

Data flow added successfully.
Data flow added successfully.
Data flow added successfully.
Invalid data flow path. Incorrect node types.
Invalid data flow path. Incorrect node types.
Invalid data flow path. Not a simple path in the graph.
nodes:
 ['H1', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'H2', 'H3']
edges:
 [('H1', 'S1'), ('H1', 'S2'), ('H1', 'S3'), ('S1', 'S4'), ('S1', 'S5'), ('S2', 'S1'), ('S2', 'S5'), ('S2', 'S6'), ('S3', 'S5'), ('S3', 'S6'), ('S4', 'H2'), ('S4', 'H3'), ('S5', 'H2'), ('S5', 'H3'), ('S6', 'H3')]
flows:
 [['H1', 'S2', 'S5', 'H3'], ['H1', 'S3', 'S6', 'H3'], ['H1', 'S1', 'S4', 'H2']]
ingress nodes:
 {'H1'}
egress nodes:
 {'H3', 'H2'}
switches:
 {'S1', 'S3', 'S6', 'S2', 'S4', 'S5'}
Alternate Paths:
['H1', 'S1', 'S5', 'H3']
['H1', 'S2', 'S1', 'S4', 'H3']
['H1', 'S2', 'S1', 'S5', 'H3']
['H1', 'S2', 'S5', 'H3']
['H1', 'S2', 'S6', 'H3']
['H1', 'S3', 'S5', 'H3']
['H1', 'S3', 'S6', 'H3']
Flow 0:
['H1', 'S1', 'S4', 'H3']
['H1', 'S1', 'S5', 'H3']
['H1', 'S2', 'S1', 'S4', 'H3']
[

## Example usage: use this class for find safe route for f3
find_safe_update = first find a alternate path of one flow in data_flow for example data_flow[2]
and second find and print whichone have not confilict with other flows in data_flow (name it Safe_updat_paths )
and third modify it to Safe_updat_paths[0] if exsist

In [ ]:
flow_index = 2
safe_update_candidates_f3 = network.find_safe_update(flow_index)

# Third, modify the path to the first safe update path that has no conflicts
if safe_update_candidates_f3:
    for new_route in safe_update_candidates_f3:
        network.modify_data_flow(flow_index, new_route)
        print(f"Modified flow {flow_index} with a safe update path: {new_route}")
    else:
        print(f"No safe update paths without conflicts found for flow {flow_index}")
else:
    print("No safe update paths found.")

Alternate paths for flow 2: ['H1', 'S1', 'S4', 'H2']

['H1', 'S1', 'S5', 'H2']
['H1', 'S2', 'S1', 'S4', 'H2']
['H1', 'S2', 'S1', 'S5', 'H2']
['H1', 'S2', 'S5', 'H2']
['H1', 'S3', 'S5', 'H2']

Safe update paths:
['H1', 'S1', 'S5', 'H2']

Data flow modified successfully.
Modified flow 2 with a safe update path: ['H1', 'S1', 'S5', 'H2']
No safe update paths without conflicts found for flow 2


#### Old Temp


In [ ]:

# First, find an alternate path for data_flow[2]
flow_index = 2
flow_path = network.data_flow[flow_index]
alternate_paths = network.find_alternate_paths(flow_path)
print(f"Alternate paths for flow {flow_index}:", flow_path ,"is:\n")
for path in alternate_paths:
    print(path)
print()

# Second, find and print the safe update paths that do not conflict with other flows
safe_update_paths = []
for i, path in enumerate(alternate_paths):
    conflicting_paths = [f for j, f in enumerate(network.data_flow) if j != flow_index and network._check_conflict_between(f, path)]
    if not conflicting_paths:
        safe_update_paths.append(path)

print("Safe update paths:")
for path in safe_update_paths:
    print(path)
print()

print("Safe update paths:")
for path in safe_update_paths:
    print(path)
print()

# Third, modify the path to the first safe update path if it exists
if safe_update_paths:
    new_path = safe_update_paths[0]
    network.modify_data_flow(flow_index, new_path)
else:
    print("No safe update paths found.")

Alternate paths for flow 2: ['H1', 'S1', 'S5', 'H2'] is:

['H1', 'S1', 'S4', 'H2']
['H1', 'S2', 'S1', 'S4', 'H2']
['H1', 'S2', 'S1', 'S5', 'H2']
['H1', 'S2', 'S5', 'H2']
['H1', 'S3', 'S5', 'H2']

Safe update paths:
['H1', 'S1', 'S4', 'H2']

Safe update paths:
['H1', 'S1', 'S4', 'H2']

Data flow modified successfully.


In [ ]:
flow_index = 2
safe_update_paths_f3 = network.find_safe_update(2)
# Third, modify the path to the first safe update path that has no conflicts
if safe_update_paths_f3:
    new_path = None
    for path in safe_update_paths_f3:
        conflicting_paths = [f for f in network.data_flow if f != path and network._check_conflict_between(f, path)]
        if not conflicting_paths:
            new_path = path
            break

    if new_path:
        network.modify_data_flow(flow_index, new_path)
        print(f"Modified flow {flow_index} with a safe update path: {new_path}")
    else:
        print(f"No safe update paths without conflicts found for flow {flow_index}")
else:
    print("No safe update paths found.")

Alternate paths for flow 2: ['H1', 'S1', 'S4', 'H2']

['H1', 'S1', 'S5', 'H2']
['H1', 'S2', 'S1', 'S4', 'H2']
['H1', 'S2', 'S1', 'S5', 'H2']
['H1', 'S2', 'S5', 'H2']
['H1', 'S3', 'S5', 'H2']

Safe update paths:
['H1', 'S1', 'S5', 'H2']

No safe update paths without conflicts found for flow 2


In [ ]:
network.print_structure()

nodes:
 ['H1', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'H2', 'H3']
edges:
 [('H1', 'S1'), ('H1', 'S2'), ('H1', 'S3'), ('S1', 'S4'), ('S1', 'S5'), ('S2', 'S1'), ('S2', 'S5'), ('S2', 'S6'), ('S3', 'S5'), ('S3', 'S6'), ('S4', 'H2'), ('S4', 'H3'), ('S5', 'H2'), ('S5', 'H3'), ('S6', 'H3')]
flows:
 [['H1', 'S2', 'S5', 'H3'], ['H1', 'S3', 'S6', 'H3'], ['H1', 'S1', 'S4', 'H2']]
ingress nodes:
 {'H1'}
egress nodes:
 {'H3', 'H2'}
switches:
 {'S1', 'S3', 'S6', 'S2', 'S4', 'S5'}


In [ ]:
# Third, modify the path to the first safe update path that has no conflicts
if safe_update_paths:
    new_path = None
    for path in safe_update_paths:
        conflicting_paths = [f for f in network.data_flow if f != path and network._check_conflict_between(f, path)]
        if not conflicting_paths:
            new_path = path
            break

    if new_path:
        network.modify_data_flow(flow_index, new_path)
        print(f"Modified flow {flow_index} with a safe update path: {new_path}")
    else:
        print(f"No safe update paths without conflicts found for flow {flow_index}")
else:
    print("No safe update paths found.")

Data flow modified successfully.
Modified flow 2 with a safe update path: ['H1', 'S1', 'S4', 'H2']


#### new temp
